In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install implicit
import numpy as np
import pandas as pd
from tqdm import trange
%cd /content/drive/My Drive/Colab Notebooks/Thesis

/content/drive/My Drive/Colab Notebooks/Thesis


In [3]:
pip install -r bpr-master/requirements.txt

  Using cached https://files.pythonhosted.org/packages/62/20/4d43e141b5bc426ba38274933ef8e76e85c7adea2c321ecf9ebf7421cedf/numpy-1.18.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/08/ec/b5dd8cfb078380fb5ae9325771146bccd4e8cad2d3e4c72c7433010684eb/pandas-1.0.1-cp36-cp36m-manylinux1_x86_64.whl
     |████████████████████████████████| 753.4MB 25kB/s 
     |████████████████████████████████| 4.0MB 34.5MB/s 
     |████████████████████████████████| 235kB 41.1MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
  Found existing installation: pandas 1.0.4
    Uninstalling pandas-1.0.4:
      Successfully uninstalled pandas-1.0.4
  Found existing installation:

In [4]:
import os
import gzip
import json
import math
import random
import pickle
import pprint
import argparse

import numpy as np
import pandas as pd


class DatasetLoader(object):
    def load(self):
        """Minimum condition for dataset:
          * All users must have at least one item record.
          * All items must have at least one user record.
        """
        raise NotImplementedError

class Hetrec(DatasetLoader):
    def __init__(self, data_dir):
        self.fpath = os.path.join('/content/drive/My Drive/Colab Notebooks/Thesis/hetrec2011-lastfm-2k', 'user_artists.dat')

    def load(self):
        # Load data
        df = pd.read_csv(self.fpath,
                         sep="\s+",
                         engine='python',
                         names=['userID', 'itemID', 'weight'])
        # TODO: Remove negative rating?
        # df = df[df['rate'] >= 3]
        return df

# class MovieLens1M(DatasetLoader):
#     def __init__(self, data_dir):
#         self.fpath = os.path.join('/content/drive/My Drive/Colab Notebooks/Thesis/data_dir', 'ratings.dat')

#     def load(self):
#         # Load data
#         df = pd.read_csv(self.fpath,
#                          sep='::',
#                          engine='python',
#                          names=['user', 'item', 'rate','time'])
#         # TODO: Remove negative rating?
#         # df = df[df['rate'] >= 3]
#         return df


# class MovieLens20M(DatasetLoader):
#     def __init__(self, data_dir):
#         self.fpath = os.path.join('Thesis/data_dir', 'ratings.dat')

#     def load(self):
#         df = pd.read_csv(self.fpath,
#                          sep=',',
#                          names=['user', 'item', 'rate'],
#                          usecols=['user', 'item'],
#                          skiprows=1)
#         return df


# class AmazonBeauty(DatasetLoader):
#     def __init__(self, data_dir, file_name='All_Beauty.json.gz'):
#         self.fpath = os.path.join(data_dir, file_name)

#     def load(self):
#         raw_list = []
#         with gzip.open(self.fpath) as f:
#             for idx, line in enumerate(f):
#                 raw_data = json.loads(line)
#                 raw_list.append({'user': raw_data['reviewerID'],
#                                  'item': raw_data['asin'],
#                                  'rate': raw_data['overall'],
#                                  'time': raw_data['unixReviewTime']})
#         df = pd.DataFrame(raw_list)
#         print('Check if any column has null value')
#         print(df.isnull().any())
#         print('Total user number: %d' % df['user'].nunique())
#         print('Total item number: %d' % df['item'].nunique())
#         print('The number of unique item per user')
#         print(df.groupby('user')['item'].nunique().value_counts())
#         print('The number of unique user per item')
#         print(df.groupby('item')['user'].nunique().value_counts())
#         return df

# class Hetrec(DatasetLoader):
#     def __init__(self, data_dir, file_name='user_artists.dat'):
#         self.fpath = os.path.join(data_dir, file_name)

#     def load(self):
#         raw_list = []
#         with gzip.open(self.fpath) as f:
#             for idx, line in enumerate(f):
#                 raw_data = json.loads(line)
#                 raw_list.append({'user': raw_data['userID'],
#                                  'item': raw_data['artistID'],
#                                  'rate': raw_data['weight'],})
#         df = pd.DataFrame(raw_list)
#         print('Check if any column has null value')
#         print(df.isnull().any())
#         print('Total user number: %d' % df['user'].nunique())
#         print('Total item number: %d' % df['item'].nunique())
#         print('The number of unique item per user')
#         print(df.groupby('user')['item'].nunique().value_counts())
#         print('The number of unique user per item')
#         print(df.groupby('item')['user'].nunique().value_counts())
#         return df

# class Gowalla(DatasetLoader):
#     """Work In Progress"""
#     def __init__(self, data_dir):
#         self.fpath = os.path.join(data_dir, 'loc-gowalla_totalCheckins.txt')

#     def load(self):
#         df = pd.read_csv(self.fpath,
#                          sep='\t',
#                          names=['user', 'time', 'latitude', 'longitude', 'item'],
#                          usecols=['user', 'item', 'time'])
#         df_size, df_nxt_size = 0, len(df)
#         while df_size != df_nxt_size:
#             # Update
#             df_size = df_nxt_size

#             # Remove user which doesn't contain at least five items to guarantee the existance of `test_item`
#             groupby_user = df.groupby('user')['item'].nunique()
#             valid_user = groupby_user.index[groupby_user >= 15].tolist()
#             df = df[df['user'].isin(valid_user)]
#             df = df.reset_index(drop=True)

#             # Remove item which doesn't contain at least five users
#             groupby_item = df.groupby('item')['user'].nunique()
#             valid_item = groupby_item.index[groupby_item >= 15].tolist()
#             df = df[df['item'].isin(valid_item)]
#             df = df.reset_index(drop=True)

#             # Update
#             df_nxt_size = len(df)

#         print('User distribution')
#         print(df.groupby('user')['item'].nunique().describe())
#         print('Item distribution')
#         print(df.groupby('item')['user'].nunique().describe())
#         return df


def convert_unique_idx(df, column_name):
    column_dict = {x: i for i, x in enumerate(df[column_name].unique())}
    # print(column_dict)
    df[column_name] = df[column_name].apply(column_dict.get)
    # df[column_name] = ''.join(map(str,df[column_name])) #converting the list into string
    # df[column_name] = np.float(df[column_name])
    # df[column_name] = df[column_name].astype('float')
    # df[column_name] = df[column_name].astype('int64')
    df[column_name] = round(df[column_name])
    # assert df[column_name].min() == 0
    # assert df[column_name].max() == len(column_dict) - 1
    return df, column_dict


def create_user_list(df, user_size):
    user_list = [list() for u in range(user_size)]
    for row in df.itertuples():
        user_list[row.userID].append(row.itemID)
    return user_list


def split_train_test(df, user_size, test_size=0.25, time_order=False):
    """Split a dataset into `train_user_list` and `test_user_list`.
    Because it needs `user_list` for splitting dataset as `time_order` is set,
    Returning `user_list` data structure will be a good choice."""
    # TODO: Handle duplicated items
    if not time_order:
        test_idx = np.random.choice(len(df), size=int(len(df)*test_size))
        train_idx = list(set(range(len(df))) - set(test_idx))
        test_df = df.loc[test_idx].reset_index(drop=True)
        train_df = df.loc[train_idx].reset_index(drop=True)
        test_user_list = create_user_list(test_df, user_size)
        train_user_list = create_user_list(train_df, user_size)
    else:
        total_user_list = create_user_list(df, user_size)
        train_user_list = [None] * len(user_list)
        test_user_list = [None] * len(user_list)
        for user, item_list in enumerate(total_user_list):
            # Choose latest item
            item_list = sorted(item_list, key=lambda x: x[0])
            # Split item
            test_item = item_list[math.ceil(len(item_list)*(1-test_size)):]
            train_item = item_list[:math.ceil(len(item_list)*(1-test_size))]
            # Register to each user list
            test_user_list[user] = test_item
            train_user_list[user] = train_item
    # # Remove time --> dont have time
    # test_user_list = [list(map(lambda x: x[1], l)) for l in test_user_list]
    # train_user_list = [list(map(lambda x: x[1], l)) for l in train_user_list]
    return train_user_list, test_user_list


def create_pair(user_list):
    pair = []
    for user, item_list in enumerate(user_list):
        pair.extend([(user, item) for item in item_list])
    return pair


def main():
    dataset = "hetrec"
    data_dir = "hetrec2011-lastfm-2k"
    output_data = os.path.join('preprocessed', 'hetrec.pickle')
    test_size = 0.25
    time_order = False
    
    if dataset == 'ml-1m':
        df = MovieLens1M(data_dir).load()
    elif dataset == 'hetrec':
        df = Hetrec(data_dir).load()
    elif dataset == 'ml-20m':
        df = MovieLens20M(data_dir).load()
    elif dataset == 'amazon-beauty':
        df = AmazonBeauty(data_dir).load()
    else:
        raise NotImplementedError
    df, user_mapping = convert_unique_idx(df, 'userID')
    df, item_mapping = convert_unique_idx(df, 'itemID')
    print('Complete assigning unique index to user and item')

    user_size = len(df['userID'].unique())
    item_size = len(df['itemID'].unique())

    train_user_list, test_user_list = split_train_test(df,
                                                       user_size,
                                                       test_size=test_size,
                                                       time_order=time_order)
    print('Complete spliting items for training and testing')

    train_pair = create_pair(train_user_list)
    print('Complete creating pair')

    dataset = {'user_size': user_size, 'item_size': item_size, 
               'user_mapping': user_mapping, 'item_mapping': item_mapping,
               'train_user_list': train_user_list, 'test_user_list': test_user_list,
               'train_pair': train_pair}
    dirname = os.path.dirname(os.path.abspath(output_data))
    os.makedirs(dirname, exist_ok=True)
    with open(output_data, 'wb') as f:
        pickle.dump(dataset, f, protocol=pickle.HIGHEST_PROTOCOL)

if __name__ == '__main__':
    main()
    # # Parse argument
    # parser = argparse.ArgumentParser()
    # parser.add_argument('--dataset',
    #                     choices=['ml-1m', 'ml-20m', 'amazon-beauty', 'gowalla','hetrec'])
    # parser.add_argument('--data_dir',
    #                     type=str,
    #                     default=os.path.join('data', 'hetrec'),
    #                     help="File path for raw data")
    # parser.add_argument('--output_data',
    #                     type=str,
    #                     default=os.path.join('preprocessed', 'hetrec.pickle'),
    #                     help="File path for preprocessed data")
    # parser.add_argument('--test_size',
    #                     type=float,
    #                     default=0.25,
    #                     help="Proportion for training and testing split")

    # args = parser.parse_args()
    # main(args)

Complete assigning unique index to user and item
Complete spliting items for training and testing
Complete creating pair


In [9]:
import os
import random
import pickle
import argparse
from collections import deque

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import IterableDataset, DataLoader, get_worker_info
from torch.utils.tensorboard import SummaryWriter


class TripletUniformPair(IterableDataset):
    def __init__(self, num_item, user_list, pair, shuffle, num_epochs):
        self.num_item = num_item
        self.user_list = user_list
        self.pair = pair
        self.shuffle = shuffle
        self.num_epochs = num_epochs

    def __iter__(self):
        worker_info = get_worker_info()
        # Shuffle per epoch
        self.example_size = self.num_epochs * len(self.pair)
        self.example_index_queue = deque([])
        self.seed = 0
        if worker_info is not None:
            self.start_list_index = worker_info.id
            self.num_workers = worker_info.num_workers
            self.index = worker_info.id
        else:
            self.start_list_index = None
            self.num_workers = 1
            self.index = 0
        return self

    def __next__(self):
        if self.index >= self.example_size:
            raise StopIteration
        # If `example_index_queue` is used up, replenish this list.
        while len(self.example_index_queue) == 0:
            index_list = list(range(len(self.pair)))
            if self.shuffle:
                random.Random(self.seed).shuffle(index_list)
                self.seed += 1
            if self.start_list_index is not None:
                index_list = index_list[self.start_list_index::self.num_workers]
                # Calculate next start index
                self.start_list_index = (self.start_list_index + (self.num_workers - (len(self.pair) % self.num_workers))) % self.num_workers
            self.example_index_queue.extend(index_list)
        result = self._example(self.example_index_queue.popleft())
        self.index += self.num_workers
        return result

    def _example(self, idx):
        u = self.pair[idx][0]
        i = self.pair[idx][1]
        j = np.random.randint(self.num_item)
        while j in self.user_list[u]:
            j = np.random.randint(self.num_item)
        return u, i, j


class BPR(nn.Module):
    def __init__(self, user_size, item_size, dim, weight_decay):
        super().__init__()
        self.W = nn.Parameter(torch.empty(user_size, dim))
        self.H = nn.Parameter(torch.empty(item_size, dim))
        nn.init.xavier_normal_(self.W.data)
        nn.init.xavier_normal_(self.H.data)
        self.weight_decay = weight_decay

    def forward(self, u, i, j):
        """Return loss value.
        
        Args:
            u(torch.LongTensor): tensor stored user indexes. [batch_size,]
            i(torch.LongTensor): tensor stored item indexes which is prefered by user. [batch_size,]
            j(torch.LongTensor): tensor stored item indexes which is not prefered by user. [batch_size,]
        
        Returns:
            torch.FloatTensor
        """
        u = self.W[u, :]
        i = self.H[i, :]
        j = self.H[j, :]
        x_ui = torch.mul(u, i).sum(dim=1)
        x_uj = torch.mul(u, j).sum(dim=1)
        x_uij = x_ui - x_uj
        log_prob = F.logsigmoid(x_uij).sum()
        regularization = self.weight_decay * (u.norm(dim=1).pow(2).sum() + i.norm(dim=1).pow(2).sum() + j.norm(dim=1).pow(2).sum())
        return -log_prob + regularization

    def recommend(self, u):
        """Return recommended item list given users.
        Args:
            u(torch.LongTensor): tensor stored user indexes. [batch_size,]
        Returns:
            pred(torch.LongTensor): recommended item list sorted by preference. [batch_size, item_size]
        """
        u = self.W[u, :]
        x_ui = torch.mm(u, self.H.t())
        pred = torch.argsort(x_ui, dim=1)
        return pred


def precision_and_recall_k(user_emb, item_emb, train_user_list, test_user_list, klist, batch=512):
    """Compute precision at k using GPU.
    Args:
        user_emb (torch.Tensor): embedding for user [user_num, dim]
        item_emb (torch.Tensor): embedding for item [item_num, dim]
        train_user_list (list(set)):
        test_user_list (list(set)):
        k (list(int)):
    Returns:
        (torch.Tensor, torch.Tensor) Precision and recall at k
    """
    # Calculate max k value
    max_k = max(klist)

    # Compute all pair of training and test record
    result = None
    for i in range(0, user_emb.shape[0], batch):
        # Create already observed mask
        mask = user_emb.new_ones([min([batch, user_emb.shape[0]-i]), item_emb.shape[0]])
        for j in range(batch):
            if i+j >= user_emb.shape[0]:
                break
            mask[j].scatter_(dim=0, index=torch.tensor(list(train_user_list[i+j])).cuda(), value=torch.tensor(0.0).cuda())
        # Calculate prediction value
        cur_result = torch.mm(user_emb[i:i+min(batch, user_emb.shape[0]-i), :], item_emb.t())
        cur_result = torch.sigmoid(cur_result)
        assert not torch.any(torch.isnan(cur_result))
        # Make zero for already observed item
        cur_result = torch.mul(mask, cur_result)
        _, cur_result = torch.topk(cur_result, k=max_k, dim=1)
        result = cur_result if result is None else torch.cat((result, cur_result), dim=0)

    result = result.cpu()
    # Sort indice and get test_pred_topk
    precisions, recalls = [], []
    for k in klist:
        precision, recall = 0, 0
        for i in range(user_emb.shape[0]):
            test = set(test_user_list[i])
            pred = set(result[i, :k].numpy().tolist())
            val = len(test & pred)
            precision += val / max([min([k, len(test)]), 1])
            recall += val / max([len(test), 1])
        precisions.append(precision / user_emb.shape[0])
        recalls.append(recall / user_emb.shape[0])
    return precisions, recalls

def main(): 
    data = os.path.join('/content/drive/My Drive/Colab Notebooks/Thesis/preprocessed', 'hetrec.pickle')
    seed = 0
    dim = 4
    lr = 1e-3
    weight_decay = 0.025
    n_epochs = 40
    batch_size = 4096
    print_every = 20
    eval_every = 100
    save_every = 10000
    model = os.path.join('output', 'bpr.pt')
    
    # Initialize seed
    np.random.seed(seed)
    torch.manual_seed(seed)

    # Load preprocess data
    with open(data, 'rb') as f:
        dataset = pickle.load(f)
        user_size, item_size = dataset['user_size'], dataset['item_size']
        train_user_list, test_user_list = dataset['train_user_list'], dataset['test_user_list']
        train_pair = dataset['train_pair']
    print('Load complete')

    # Create dataset, model, optimizer
    dataset = TripletUniformPair(item_size, train_user_list, train_pair, True, n_epochs)
    loader = DataLoader(dataset, batch_size=batch_size, num_workers=16)
    model = BPR(user_size, item_size, dim, weight_decay).cuda()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    writer = SummaryWriter()

    # Training
    smooth_loss = 0
    idx = 0
    for u, i, j in loader:
        optimizer.zero_grad()
        loss = model(u, i, j)
        loss.backward()
        optimizer.step()
        writer.add_scalar('train/loss', loss, idx)
        smooth_loss = smooth_loss*0.99 + loss*0.01
        if idx % print_every == (print_every - 1):
            print('loss: %.4f' % smooth_loss)
            # print(idx)
            # print(idx % eval_every)
            # print(eval_every - 1)
        if idx % eval_every == (eval_every - 1):
            plist, rlist = precision_and_recall_k(model.W.detach(),
                                                    model.H.detach(),
                                                    train_user_list,
                                                    test_user_list,
                                                    klist=[1, 5, 10])
            print('P@1: %.4f, P@5: %.4f P@10: %.4f, R@1: %.4f, R@5: %.4f, R@10: %.4f' % (plist[0], plist[1], plist[2], rlist[0], rlist[1], rlist[2]))
            writer.add_scalars('eval', {'P@1': plist[0],
                                                    'P@5': plist[1],
                                                    'P@10': plist[2]}, idx)
            writer.add_scalars('eval', {'R@1': rlist[0],
                                                'R@5': rlist[1],
                                                'R@10': rlist[2]}, idx)
        if idx % save_every == (save_every - 1):
            dirname = os.path.dirname(os.path.abspath(model))
            os.makedirs(dirname, exist_ok=True)
            torch.save(model.state_dict(), model)
        idx += 1
        
if __name__ == '__main__':
  main()

Load complete
loss: 517.0642
loss: 939.9053
loss: 1285.6853
loss: 1568.3467
loss: 1799.0814
P@1: 0.0961, P@5: 0.0771 P@10: 0.0672, R@1: 0.0091, R@5: 0.0358, R@10: 0.0583
loss: 1986.6620
loss: 2137.7771
loss: 2257.7046
loss: 2351.0508
loss: 2421.0427
P@1: 0.1801, P@5: 0.1301 P@10: 0.1128, R@1: 0.0174, R@5: 0.0608, R@10: 0.0975
loss: 2470.8005
loss: 2502.5469
loss: 2519.3704
loss: 2523.1741
loss: 2515.5540
P@1: 0.1902, P@5: 0.1336 P@10: 0.1122, R@1: 0.0175, R@5: 0.0628, R@10: 0.0971
loss: 2499.3455
loss: 2474.8379
loss: 2444.5378
loss: 2408.9631
loss: 2368.8203
P@1: 0.1823, P@5: 0.1291 P@10: 0.1096, R@1: 0.0169, R@5: 0.0605, R@10: 0.0948
loss: 2325.8193
loss: 2282.6333
loss: 2238.3979
loss: 2193.4138
loss: 2150.3726
P@1: 0.1828, P@5: 0.1241 P@10: 0.1080, R@1: 0.0169, R@5: 0.0579, R@10: 0.0933
loss: 2107.9331
loss: 2067.7373
loss: 2027.8967
loss: 1991.0612
loss: 1956.2156
P@1: 0.1722, P@5: 0.1214 P@10: 0.1074, R@1: 0.0160, R@5: 0.0568, R@10: 0.0928
loss: 1921.9966
loss: 1891.8849
loss: 18